Taken from
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS

In [11]:
import numpy as np
import pandas as pd
import os, re

from datetime import datetime

from astropy.table import Table
from astroquery.utils.tap.core import TapPlus
from astroquery.utils.tap.core import Tap

import astropy.coordinates as coord
from astropy.io.votable import parse
from astropy.coordinates import SkyCoord

import astropy.units as un

test_df = pd.read_csv("./proxima_cen_b_test/PS_2024.05.09_04.24.22.csv")
test_df

,pl_name,hostname,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_pubdate,disc_facility,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,Proxima Cen b,Proxima Cen,Gaia DR2 5853498713160606720,1,3,1,Radial Velocity,2016,2016-08,European Southern Observatory,...,-0.1,4.384,0.033,-0.033,8.95361,0.000918,-0.000918,2023-05-15,2022-02,2023-05-15


intial non-proper motion corrected RA and DEC

In [12]:
test_data = test_df.iloc[0]
test_ra = test_data['ra']
test_dec = test_data['dec']
test_ra, test_dec

(217.3934657, -62.6761821)

In [18]:
import casda_util 
from astroquery.casda import Casda

username = "brian.w.tjahjadi@gmail.com"
casda = Casda()
casda.login(username=username)

casda_util.casda_search(test_ra,test_dec, 3, casda=casda, refresh=False, debug=True)

INFO: Authenticating brian.w.tjahjadi@gmail.com on CASDA ... [astroquery.casda.core]
INFO: Authentication successful! [astroquery.casda.core]
-- Target source <SkyCoord (ICRS): (ra, dec) in deg
    (217.3934657, -62.6761821)>  
-- Beginning pubdat retrieval
-- pubdat files retrieved:
 0       selavy-image.i.RACS_2221+14.SB29572.cont.taylo...
1       selavy-image.i.RACS_1041-18.SB21847.cont.taylo...
2       selavy-image.i.RACS_1348+28.SB20477.cont.taylo...
3       selavy-image.i.RACS_1608+37.SB20536.cont.taylo...
4       selavy-image.i.RACS_1902-23.SB21440.cont.taylo...
                              ...                        
9964    selavy-image.i.RACS_1708-60.SB58924.cont.taylo...
9965    selavy-image.i.RACS_1629-55.SB58917.cont.taylo...
9966    selavy-image.i.RACS_1900-28.SB59068.cont.taylo...
9967    selavy-image.i.RACS_1542-37.SB58865.cont.taylo...
9968    selavy-image.i.WALLABY_0857-00B.SB59434.cont.t...
Name: filename, Length: 9969, dtype: object
-- reduced pubdat files retrieve

Extracting `t_min` and `t_max` from the catalogue file with the least separation:
`selavy-image.i.VAST_1453-62.SB50301.cont.taylor.0.restored.conv.components.xml`

In [29]:
pubdat_df = pd.read_csv("./casda_cache/pubdat-2024-05-06.csv")
filename = "selavy-image.i.VAST_1453-62.SB50301.cont.taylor.0.restored.conv.components.xml"
filtered_rows = pubdat_df[pubdat_df['filename'] == filename]
t_min_values = filtered_rows['t_min']
t_max_values = filtered_rows['t_max']

print("t_min: ",t_min_values.values)
print("t_max: ",t_max_values.values)

t_min:  [60097.53744213]
t_max:  [60097.54572917]


In [31]:
import proper_motion
test_df['epoch'] = t_max_values.values[0]
proper_corrected_test_df = proper_motion.proper_correction(test_df)
proper_corrected_test_df

,pl_name,hostname,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_pubdate,disc_facility,...,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate,epoch,ra_corrected,dec_corrected
0,Proxima Cen b,Proxima Cen,Gaia DR2 5853498713160606720,1,3,1,Radial Velocity,2016,2016-08,European Southern Observatory,...,-0.033,8.95361,0.000918,-0.000918,2023-05-15,2022-02,2023-05-15,60097.545729,217.375351,-62.674488


In [35]:
proper_corrected_ra  = proper_corrected_test_df['ra_corrected'].values[0]
proper_corrected_dec = proper_corrected_test_df['dec_corrected'].values[0]
print(proper_corrected_ra, proper_corrected_dec)

217.3753505019197 -62.67448809582359


In [36]:
casda_util.casda_search(proper_corrected_ra,proper_corrected_dec, 3, casda=casda, refresh=False, debug=True)

-- Target source <SkyCoord (ICRS): (ra, dec) in deg
    (217.3753505, -62.6744881)>  
-- Beginning pubdat retrieval
-- pubdat files retrieved:
 0       selavy-image.i.RACS_2221+14.SB29572.cont.taylo...
1       selavy-image.i.RACS_1041-18.SB21847.cont.taylo...
2       selavy-image.i.RACS_1348+28.SB20477.cont.taylo...
3       selavy-image.i.RACS_1608+37.SB20536.cont.taylo...
4       selavy-image.i.RACS_1902-23.SB21440.cont.taylo...
                              ...                        
9964    selavy-image.i.RACS_1708-60.SB58924.cont.taylo...
9965    selavy-image.i.RACS_1629-55.SB58917.cont.taylo...
9966    selavy-image.i.RACS_1900-28.SB59068.cont.taylo...
9967    selavy-image.i.RACS_1542-37.SB58865.cont.taylo...
9968    selavy-image.i.WALLABY_0857-00B.SB59434.cont.t...
Name: filename, Length: 9969, dtype: object
-- reduced pubdat files retrieved:
 0       selavy-image.i.RACS_2221+14.SB29572.cont.taylo...
1       selavy-image.i.RACS_1041-18.SB21847.cont.taylo...
2       selavy-image.i

2 Matches were found for proxima centauri B.